In [499]:
import schillinger.rhythm as SR
import schillinger.pitch as SP
import random

In [134]:
#notes
from music21 import *
#from itertools import chain
%load_ext music21.ipython21

def convert2Dto1D(data):
    flat_array = []
    for d in data:
        if isinstance(d[0], list) or isinstance(d[0], tuple):
            for l in d:
                
                flat_array.append(l)
        else:
            flat_array.append(d)
    return flat_array
    

def showSplittedNotes(length, data, time_meter, note_sequece, duration_divider):
    attacks = convert2Dto1D(data)
    #TS = str(int(len(clock))) +'/'+ str(int((clock[0].duration.quarterLength*16)))
    TS = str(time_meter[0])+'/'+str(time_meter[1])
    
    s = stream.Score(id='mainScore')
    
    for sequences in attacks:
        p0 = stream.Part(id='part0')
        p0.timeSignature = meter.TimeSignature(TS)
        tones = []
        note_counter = 0
        for i in range(length):
            nota = pitch.Pitch(note_sequece[note_counter%len(note_sequece)])
            note_counter += 1
            #nota.octave = 4
            nota = note.Note(nota.nameWithOctave)
            nota.duration.quarterLength = sequences[i%len(sequences)]*duration_divider
            tones.append(nota)
            
        p0.append(tones)
        s.insert(0, p0)
    return s

def showSplittedNotesWhole(length, data, time_meter, note_sequece, duration_divider):
    attacks = convert2Dto1D(data)
    #TS = str(int(len(clock))) +'/'+ str(int((clock[0].duration.quarterLength*16)))
    TS = str(time_meter[0])+'/'+str(time_meter[1])
    
    s = stream.Score(id='mainScore')
    
    for sequences in attacks:
        p0 = stream.Part(id='part0')
        p0.timeSignature = meter.TimeSignature(TS)
        tones = []
        note_counter = 0
        for i in range(length):
            nota = pitch.Pitch(note_sequece[note_counter%len(note_sequece)])
            note_counter += 1
            #nota.octave = 4
            nota = note.Note(nota.nameWithOctave)
            nota.duration.quarterLength = sequences[i%len(sequences)]*duration_divider
            tones.append(nota)
            
        p0.append(tones)
        s.insert(0, p0)
    return s


The music21.ipython21 extension is already loaded. To reload it, use:
  %reload_ext music21.ipython21


In [3]:
#help(schillinger)

In [352]:
a = 5
b = 3
fraction = [a,b]

sr = SR.Type_I(fraction)
_sr_ = SR.Type_II(fraction)
sr.resultant, _sr_.resultant

([3, 2, 1, 3, 1, 2, 3], [3, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 3])

In [353]:
# expansion tests bach
scale = ["C","D","E-","F","G","A-","B-"]
# Bach Fuga Cux
notes = ["G","E-","F","G","C", "F", "E-", "D", "E-"]
spg = SP.PitchGroup()
seq = spg.translate_notes_to_expansions(notes, scale)
#seq, seq[5]

In [387]:

note_sequence = seq[2] # 5th expansion
#rotation
cr = spg.clock_rotation(note_sequence)
ns = cr[0] # 2th rotation
ns

['A-', 'B-', 'E-', 'A-', 'C', 'E-', 'B-', 'F', 'B-']

In [388]:
result = _sr_.resultant
fraction = 5/3
r = SR.slice_by_fraction(result, .2)
print(r)
s = showSplittedNotes(len(r[1]), [r[1]], [4,4], ns, 1/2)

sp = midi.realtime.StreamPlayer(s)
sp.play()
#s.show('midi')
#s.show('musicxml')


[[3, 2, 1], [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 3]]


In [367]:
1.68/3.14

0.535031847133758

In [ ]:
# Pitch Second Group
rhythm_attacks = sr.resultant

def two_unit_scales(tone):
    scale_array = []
    for i in range(12):
        count = (tone+i)%12
        if count != tone:
            scale_array.append(count)
    return scale_array


A = 0 ##### Start tone
scales = two_unit_scales(A)
B = scales[4] ##### Interval
MF = [A,B] ##### sequence array (could be reused and start there!!!)

def melodic_forms_definition(units):
    return list(itertools.permutations(units))
MF1 = melodic_forms_definition(MF)
print(MF1)

def clock_rotation(units):
    # Rotate Lists
    def rotate(l, x):
      return l[-x % len(l):] + l[:-x % len(l)]
    #clock
    l = []
    for i,e in enumerate(units):
        l.append(rotate(units, i))
        #print(rotate(units, i))
    return l
def counter_clock_rotation(units):
    # Rotate Lists
    def rotate(l, x):
      return l[-x % len(l):] + l[:-x % len(l)]
    l = []
    #counterclock
    for i,e in enumerate(units):
        l.append(rotate(units, -i))
        #print(rotate(units, -i))
    return l

MFR = clock_rotation(MF)
print("---",MFR)
###----------> Higher Order??? warning!!!
def melodic_forms_combination(units):
    return list(itertools.permutations(units))
#MF2= melodic_forms_combination(MF1)
#print(MF2)
#### <------------------------------------
print()
melodic_coeficients = [1,1,1,1,1] ##### f.e. 2a+b
def max_coefficients_of_reccurence(coeficients, mel_form):
    #get the higher number and iterate over both lists whith %
    max_num = max(len(coeficients), len(mel_form))
    temp = []
    for i in range(max_num):
        temp.append(mel_form[i%len(mel_form)]*coeficients[i%len(coeficients)])
    return temp

def coefficients_of_reccurence(coeficients, mel_form):
    temp = []
    for i,e in enumerate(coeficients):
        temp.append([mel_form[i%len(mel_form)]]*e)
    
    return temp

melodic_continuity = coefficients_of_reccurence(melodic_coeficients,MF1)

def convert_to_1D(sequence, levels_of_depth):
    for i in range(levels_of_depth):
        sequence = list(itertools.chain(*sequence))
    return sequence

tone_sequence = convert_to_1D(melodic_continuity, 2)

#Synchronization!!!! check ich ned!!
'''def synchronization(melodic_attacks, rhythmic_attacks, modulo):
    if len(rhythm_attacks) % modulo == 0:
        
        return int((melodic_attacks*rhythmic_attacks)/modulo)
    else:
        return int(melodic_attacks*rhythmic_attacks)

attack_synchronization = synchronization(len(tone_sequence),len(rhythm_attacks), len(tone_sequence)/len(MF1))
print(attack_synchronization)'''

attack_synchronization = int(len(tone_sequence)*len(rhythm_attacks))

def convert_to_note_sequence(tone_sequence, attacks):
    new_seq = []
    for i in range(attacks):
        new_seq.append(tone_sequence[i%len(tone_sequence)])
    return new_seq

whole_sequence = convert_to_note_sequence(tone_sequence, attack_synchronization)

s = showSplittedNotes(len(whole_sequence),[[2]], [5,4], whole_sequence, 1/4)
s.show('midi')
s.show()

In [250]:
result = sr.resultant
fraction = 2/4
SR.slice_by_fraction(result, fraction)

[[3, 1, 2], [2, 1, 3]]

(['C2', 'D2', 'C2', 'G#2', 'E2', 'D2', 'E2', 'B-2'],
 ['D3', 'B-3', 'G3', 'E3', 'B-3', 'E3', 'C3', 'C3'])

In [572]:

def showMultipleScore(note_sequence, rhythm, length, time_meter, duration_divider):
    #attacks = convert2Dto1D(data)
    #TS = str(int(len(clock))) +'/'+ str(int((clock[0].duration.quarterLength*16)))
    TS = str(time_meter[0])+'/'+str(time_meter[1])
    
    s = stream.Score(id='mainScore')
    
    for es, sequences in enumerate(note_sequence):
        
        #print(sequences)
        p0 = stream.Part(id='part0')
        p0.timeSignature = meter.TimeSignature(TS)
        tones = []
        
        flag = 0
        for i,e  in enumerate(sequences):
            dd = i%4==0
            if dd:
                flag = 0
            else:
                flag += 1
            #note_counter += 1
            
            if es == 0:
                #print(flag)
                if flag >= 2:
                    nota = pitch.Pitch(e)
                    nota.octave = 5
                    nota = note.Note(nota.nameWithOctave)
                    nota.duration.quarterLength = 1*duration_divider
                else:
                    
                    nota = note.Rest()
                    nota.duration.quarterLength = 0.5*duration_divider
            
            else:
                nota = pitch.Pitch(e)
                nota = note.Note(nota.nameWithOctave)
                nota.duration.quarterLength = 3*duration_divider
            
            
            
            tones.append(nota)
            
        p0.append(tones)
        s.insert(0, p0)
    return s


def showMultipleScore_rythm(note_sequence, rhythm, length, time_meter, duration_divider):
    
    print(rhythm)
    #rhythm = convert2Dto1D(rhythm)
    #TS = str(int(len(clock))) +'/'+ str(int((clock[0].duration.quarterLength*16)))
    TS = str(time_meter[0])+'/'+str(time_meter[1])
    
    s = stream.Score(id='mainScore')
    
    for es, sequences in enumerate(note_sequence):
        
        #print(sequences)
        p0 = stream.Part(id='part0')
        p0.timeSignature = meter.TimeSignature(TS)
        tones = []
        razlika = len(note_sequence[0])/len(note_sequence[1])
        flag = 0
        for i,e  in enumerate(sequences):
            dd = i%4==0
            if dd:
                flag = 0
            else:
                flag += 1
            #note_counter += 1
            
            if es == 0:
                #print(flag)
                
                nota = pitch.Pitch(e)
                nota.octave = 5
                nota = note.Note(nota.nameWithOctave)
                nota.duration.quarterLength = rhythm[i%len(rhythm)]
                
            
            else:
                nota = pitch.Pitch(e)
                nota = note.Note(nota.nameWithOctave)
                nota.duration.quarterLength = 6
            
            
            
            tones.append(nota)
            
        p0.append(tones)
        s.insert(0, p0)
    return s

def showMultipleScore_MelodyHarmony(note_sequence, rhythm, length, time_meter, duration_divider):
    #attacks = convert2Dto1D(data)
    #TS = str(int(len(clock))) +'/'+ str(int((clock[0].duration.quarterLength*16)))
    TS = str(time_meter[0])+'/'+str(time_meter[1])
    
    s = stream.Score(id='mainScore')
    p0 = stream.Part(id='part0')
    p0.timeSignature = meter.TimeSignature(TS)
    p1 = stream.Part(id='part0')
    p1.timeSignature = meter.TimeSignature(TS)
    for es, sequences in enumerate(note_sequence):
        
        
        tones = []
        tones2 = []
        
        nota = pitch.Pitch(sequences[0])
        nota.octave = 4
        nota = note.Note(nota.nameWithOctave)
        nota.duration.quarterLength = 4

        tones.append(nota)
        #print(sequences[1])
        chord_notes = []
        for i,e  in enumerate(sequences[1]):
            
            nota = pitch.Pitch(e)
            nota.octave = 2+i
            nota = note.Note(nota.nameWithOctave)
            nota.duration.quarterLength = 4
            chord_notes.append(nota)
        c = chord.Chord(chord_notes)
        tones2.append(c)
            
        p0.append(tones)
        p1.append(tones2)
    s.insert(0, p0)
    s.insert(0, p1)
    return s


In [574]:
# expansion tests mozart
init_scale = ["D","E","F","G","G#","A","B-","C","C#"]
# Bach Fuga Cux
notes = ["D","F","C#","D", "F","A","G#","A",
         "E","G","D","C", "G","C#","C","B-", 
         "F","D","A","D", "G","E","B-","G", 
         "A","D","E","F", "A","G","A","C#"]
spg = SP.PitchGroup()
seq = spg.translate_notes_to_expansions(notes, init_scale)

#init_scale = ["D","E","F","G","G#","A","B-","C","C#"]
scale_expansion = spg.expansions(init_scale)
scale = scale_expansion[1]
def chordify_scale(scala, voices):
    new_list=[]
    for r in range(len(list_stuff)):
        l = []
        for d in range(voices):
            l.append(list_stuff[(r+d)%len(list_stuff)])
        new_list.append(l)
    return new_list


note_sequence = seq[3] # 1th expansion
chordified_scale = chordify_scale(init_scale,3)

def harmonize(note_seq, chordified_scale):
    # erstma mit choice
    array = []
    for i, note in enumerate(note_seq):
        temp_cord = []
        for chord in chordified_scale:
            if note in chord:
                temp_cord.append(chord)
        array.append([note,random.choice(temp_cord)])
    return array

harmonized = harmonize(note_sequence,chordified_scale)

def clean_harmony(harmonized_part):
    cleaned_list = []
    for i,p in enumerate(harmonized_part):
        cl = []
        for l in p[1]:
            if l != p[0]:
                cl.append(l)
        cleaned_list.append([p[0],cl])
    return cleaned_list

cleaned_harmony = clean_harmony(harmonized)

s = showMultipleScore_MelodyHarmony(cleaned_harmony, [1], len(cleaned_harmony), [4,4], 1/1)
#sp = midi.realtime.StreamPlayer(s)
#sp.play()
#s.show('midi')
s.show('musicxml')
#s.show()

In [348]:
#note_sequence = seq[1] # 1th expansion
#rotation

cr = spg.clock_rotation(note_sequence)
ns = cr[0] # 0th rotation
print(ns)
low_notes, low_notes_2 = ns[0::4],ns[1::4]
low_notes = [x+str(4) for x in low_notes]
low_notes_2 = [x+str(2) for x in low_notes_2]
print(low_notes, low_notes_2)


['E', 'B-', 'E', 'C', 'D', 'G#', 'C', 'D', 'G#', 'E', 'C#', 'E', 'F', 'B-', 'D', 'A', 'B-', 'C', 'A', 'G', 'G#', 'D', 'E', 'D', 'B-', 'F', 'G', 'B-', 'E', 'D', 'F', 'G#']


(['E4', 'D4', 'G#4', 'F4', 'B-4', 'G#4', 'B-4', 'E4'],
 ['B-2', 'G#2', 'E2', 'B-2', 'C2', 'D2', 'F2', 'D2'])

In [351]:
## result = sr.resultant
fraction = 2/4
sliced_rhytm = SR.slice_by_fraction(result, fraction)
whole_seq = [ns,low_notes, low_notes_2]
#whole_seq = [ns,[], []]
s = showMultipleScore_rythm(whole_seq, [2,1,1,2], len(ns), [3,4], 1/2)

#sp = midi.realtime.StreamPlayer(s)
#sp.play()
#s.show('midi')
s.show('musicxml')
#s.show()

[2, 1, 1, 2]


In [493]:
#
init_scale = ["D","E","F","G","G#","A","B-","C","C#"]
scale_expansion = spg.expansions(init_scale)
scale = scale_expansion[1]
def chordify_scale(scala, voices):
    new_list=[]
    for r in range(len(list_stuff)):
        l = []
        for d in range(voices):
            l.append(list_stuff[(r+d)%len(list_stuff)])
        new_list.append(l)
    return new_list

chordify_scale(scale,2)

[['D', 'F'],
 ['F', 'G#'],
 ['G#', 'B-'],
 ['B-', 'C#'],
 ['C#', 'E'],
 ['E', 'G'],
 ['G', 'A'],
 ['A', 'C'],
 ['C', 'D']]

[['D', 'F', 'G#'],
 ['F', 'G#', 'B-'],
 ['G#', 'B-', 'C#'],
 ['B-', 'C#', 'E'],
 ['C#', 'E', 'G'],
 ['E', 'G', 'A'],
 ['G', 'A', 'C'],
 ['A', 'C', 'D'],
 ['C', 'D', 'F']]